In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dateutil.parser
import seaborn as sns
from IPython.display import Image
import os
current_dataset = '../data/pkl/df_pluto-cat-2013-2017.pkl'


In [10]:
df_pluto = pd.read_pickle('../data/pkl/df_pluto-cat-2013-2017.pkl')

> Year built feature

In [11]:
def check_block_new_bldg(year):
    if year >= 2002:
        return 1
    else:
        return 0

# Create feature based on whether block contains a new building
df_pluto['new_bldg_prob'] = df_pluto['YearBuilt'].apply(check_block_new_bldg)
df_pluto['new_bldg_prob'].value_counts()


0    262469
1     11963
Name: new_bldg_prob, dtype: int64

> Add Lot Count and Density Features

In [12]:
def lot_resident_ratio(row):
    if row['UnitsTotal'] == 0:
        return 0
    return row['LotCount']/row['UnitsTotal']


# Create column with total number of lots on block
df_pluto['LotCount'] = df_pluto.groupby(["Block"])["Lot"].transform("count")
df_pluto['LotCount'] = df_pluto['LotCount'].astype(float)

# Create column with amount of lots/ number of residents
df_pluto['LotUnitRatio'] = df_pluto.apply(lot_resident_ratio,axis=1)

# TODO: Amount of lots/ number of residents as binary value
#df_pluto['LotUnitBinary'] = 1 if df_pluto['LotUnitRatio'] >= .5 else 0
df_pluto.head(5)

,AssessLand,AssessTot,BldgArea,BldgClass,BldgDepth,BldgFront,Block,BsmtCode,BuiltFAR,CB2010,...,Cat_ZipCode_11237.0,Cat_ZipCode_11238.0,Cat_ZipCode_11239.0,Cat_ZipCode_11241.0,Cat_ZipCode_11249.0,Cat_ZipCode_11251.0,Cat_ZipCode_11416.0,new_bldg_prob,LotCount,LotUnitRatio
0,1371105,1371105,0,V1,0.0,0.0,1,5.0,0.00,NaN,...,0,0,0,0,0,0,0,0,2.0,0.0
1,834300,10156950,163894,O6,195.0,88.0,1,0.0,8.33,2000.0,...,0,0,0,0,0,0,0,0,2.0,1.0
2,0,0,0,U4,0.0,0.0,3,5.0,0.00,3002.0,...,0,0,0,0,0,0,0,0,3.0,0.0
3,21600,21600,0,T2,0.0,0.0,3,5.0,0.00,NaN,...,0,0,0,0,0,0,0,0,3.0,0.0
4,1854,1854,0,T2,0.0,0.0,3,5.0,0.00,NaN,...,0,0,0,0,0,0,0,0,3.0,0.0


> Add Lot Count and Density Features 

In [13]:
def lot_resident_ratio(row):
    if row['UnitsTotal'] == 0:
        return 0
    return row['LotCount']/row['UnitsTotal']


# Create column with total number of lots on block
df_pluto['LotCount'] = df_pluto.groupby(["Block"])["Lot"].transform("count")
df_pluto['LotCount'] = df_pluto['LotCount'].astype(float)

# Create column with amount of lots/ number of residents
df_pluto['LotUnitRatio'] = df_pluto.apply(lot_resident_ratio,axis=1)

# Amount of lots/ number of residents as binary value
#df_pluto['LotUnitBinary'] = 1 if df_pluto['LotUnitRatio'] >= .5 else 0
df_pluto.head(5)

,AssessLand,AssessTot,BldgArea,BldgClass,BldgDepth,BldgFront,Block,BsmtCode,BuiltFAR,CB2010,...,Cat_ZipCode_11237.0,Cat_ZipCode_11238.0,Cat_ZipCode_11239.0,Cat_ZipCode_11241.0,Cat_ZipCode_11249.0,Cat_ZipCode_11251.0,Cat_ZipCode_11416.0,new_bldg_prob,LotCount,LotUnitRatio
0,1371105,1371105,0,V1,0.0,0.0,1,5.0,0.00,NaN,...,0,0,0,0,0,0,0,0,2.0,0.0
1,834300,10156950,163894,O6,195.0,88.0,1,0.0,8.33,2000.0,...,0,0,0,0,0,0,0,0,2.0,1.0
2,0,0,0,U4,0.0,0.0,3,5.0,0.00,3002.0,...,0,0,0,0,0,0,0,0,3.0,0.0
3,21600,21600,0,T2,0.0,0.0,3,5.0,0.00,NaN,...,0,0,0,0,0,0,0,0,3.0,0.0
4,1854,1854,0,T2,0.0,0.0,3,5.0,0.00,NaN,...,0,0,0,0,0,0,0,0,3.0,0.0


> Bucket units by every x floor count

In [14]:
#df_pluto.groupby(["Block"])["Lot"].count().reset_index(name="count")
def height_bucket(row):
    if row['NumFloors'] == 0:
        return 0
    return int(row['NumFloors']/3)

# Amount of lots/ number of residents
df_pluto['FloorCountBucket'] = df_pluto.apply(height_bucket,axis=1)

# bucket by every 3 floors
df_pluto['FloorCountBucket'].value_counts()

0     192935
1      77296
2       3627
3        184
4        172
5         75
6         40
7         30
8         25
10        14
9         11
11         5
12         4
13         4
14         2
17         2
25         1
15         1
18         1
19         1
20         1
39         1
Name: FloorCountBucket, dtype: int64

> Save feature dataframe

In [15]:
df_pluto.to_pickle('../data/pkl/df_pluto-cat-feat-2013-2017.pkl')